In [5]:
# mpu6050_accel.py
import tdwf
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Registers (integers)
PWR_MGMT1 = 0x6B
PWR_MNGMT2 = 0x6C
WHO_AM_I = 0x75
SMPLRT_DIV = 0x19
CONFIG = 0x1A
GYRO_CONFIG = 0x1B
ACCEL_CONFIG = 0x1C
ACCEL_XOUT_H = 0x3B  # burst read from here 6 bytes: XH XL YH YL ZH ZL
SAD = 0x68

def twos_comp(val, bits=16):
    if val & (1 << (bits - 1)):
        val -= 1 << bits
    return val

def convert_accel_word(high, low, scale_div=16384.0):
    raw = (high << 8) | low
    raw = twos_comp(raw, 16)
    return raw / scale_div, raw

def main():
    ad2 = tdwf.AD2()
    try:
        ad2.vdd = 3.3
        ad2.power(True)
        i2c = tdwf.I2Cbus(ad2.hdwf)
        devs = i2c.scan()
        print("I2C devices:", [hex(d) for d in devs])

        sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

        # Reset + configuration
        sht.write([PWR_MGMT1, 0x80])  # device reset
        time.sleep(0.5)
        sht.write([PWR_MGMT1, 0x00])  # wake up
        time.sleep(0.05)
        sht.write([PWR_MNGMT2, 0x00])  # enable accel & gyro
        time.sleep(0.01)
        sht.write([CONFIG, 0x03])      # DLPF
        time.sleep(0.01)
        sht.write([SMPLRT_DIV, 0x04])  # sample rate divider
        time.sleep(0.01)
        sht.write([GYRO_CONFIG, 0x00]) # gyro FS = ±250 dps (not used here)
        time.sleep(0.01)
        sht.write([ACCEL_CONFIG, 0x00])# accel FS = ±2g -> sensitivity 16384 LSB/g
        time.sleep(0.01)

        # Verify WHO_AM_I
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("WHO_AM_I mismatch: 0x%02x" % sht.vals[0])
            return

        # Plot setup
        fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8), sharex=True)
        plt.subplots_adjust(hspace=0.4)
        start = time.time()
        times = []
        ax1.set_ylabel("X [g]"); ax1.grid(True)
        ax2.set_ylabel("Y [g]"); ax2.grid(True)
        ax3.set_ylabel("Z [g]"); ax3.set_xlabel("Time [s]"); ax3.grid(True)
        lineX, = ax1.plot([], [], marker='o', linestyle='')
        lineY, = ax2.plot([], [], marker='o', linestyle='')
        lineZ, = ax3.plot([], [], marker='o', linestyle='')

        running = True
        def on_key(event):
            nonlocal running
            if event.key == 'escape':
                running = False
        fig.canvas.mpl_connect("key_press_event", on_key)
        plt.show(block=False)

        ACCEL_SCALE = 16384.0  # LSB/g for ±2g per register map
        AX, AY, AZ = [], [], []

        while running:
            # burst read 6 bytes from ACCEL_XOUT_H
            sht.writeread([ACCEL_XOUT_H], 6)
            hx, lx, hy, ly, hz, lz = sht.vals[0:6]
            ax_g, rawx = convert_accel_word(hx, lx, ACCEL_SCALE)
            ay_g, rawy = convert_accel_word(hy, ly, ACCEL_SCALE)
            az_g, rawz = convert_accel_word(hz, lz, ACCEL_SCALE)

            t = time.time() - start
            times.append(t)
            AX.append(ax_g); AY.append(ay_g); AZ.append(az_g)

            # update plot data
            lineX.set_data(times, AX)
            lineY.set_data(times, AY)
            lineZ.set_data(times, AZ)

            # autoscale
            for ax in (ax1, ax2, ax3):
                ax.relim()
                ax.autoscale_view()

            fig.canvas.draw()
            fig.canvas.flush_events()

            time.sleep(0.02)  # ~50 Hz loop; effective sample rate depends on SMPLRT_DIV and DLPF

    finally:
        print("Cleaning up hardware...")
        try:
            plt.close('all')
        except:
            pass
        ad2.power(False)
        ad2.close()
        print("AD2 Power Off. Process safely terminated.")

if __name__ == "__main__":
    main()


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
I2C devices: ['0x68']
Cleaning up hardware...
Dispositivo disconnesso.
AD2 Power Off. Process safely terminated.


KeyboardInterrupt: 